<a href="https://colab.research.google.com/github/kirtigupta25/Predictive_assignments/blob/main/sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN, BorderlineSMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
url = "https://github.com/AnjulaMehto/Sampling_Assignment/raw/main/Creditcard_data.csv"
df = pd.read_csv(url)


In [3]:
target_variable = df.columns[-1]

In [4]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(df.drop(target_variable, axis=1), df[target_variable])

In [5]:
X = X_resampled
y = y_resampled


In [6]:
def topsis_score(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return ((accuracy + precision + recall + f1) / 4) * 100


In [7]:
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),  # Set probability=True for AUC-ROC
    'LogisticRegression': LogisticRegression(random_state=42),
    'KNN': KNeighborsClassifier(),
    'GradientBoosting': GradientBoostingClassifier(random_state=42)
}

In [8]:
samplings = {
    'RandomUnderSampling': RandomUnderSampler(random_state=42),
    'RandomOverSampling': RandomOverSampler(random_state=42),
    'SMOTE': SMOTE(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'BorderlineSMOTE': BorderlineSMOTE(random_state=42)
}

In [9]:
results_list = []

In [10]:
for model_name, model in models.items():
    for sampling_name, sampling in samplings.items():
        X_resampled, y_resampled = sampling.fit_resample(X, y)
        X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calculate TOPSIS score
        topsis = topsis_score(y_test, y_pred)

        results_list.append({
            'Model': model_name,
            'Sampling Technique': sampling_name,
            'TOPSIS Score': topsis
        })


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [11]:
results_df = pd.DataFrame(results_list)

In [12]:
table_df = results_df.pivot(index='Model', columns='Sampling Technique', values='TOPSIS Score')

In [13]:
excel_file_path = "sampling_results.xlsx"
table_df.to_excel(excel_file_path)

print(f"Table with TOPSIS scores saved to {excel_file_path}")

Table with TOPSIS scores saved to sampling_results.xlsx
